In [54]:
import json, pycm, os, re, pandas as pd
from src.utils import  confusion_matrix, display_summary_stats

In [55]:
wikidata_runs = [
    ("google/gemma-7b-it", json.load(open('experiments/gemma-7b-it-wikidata.json', 'r'))),
    ("gpt-3.5-turbo", json.load(open('experiments/gpt-3.5-turbo-wikidata.json', 'r'))),
    ("gpt-4-0125-preview", json.load(open('experiments/gpt-4-0125-preview-wikidata.json', 'r'))),
    ("mistralai/Mistral-7B-Instruct-v0.2", json.load(open('experiments/Mistral-7B-Instruct-v0.2-wikidata.json', 'r'))),
    ("mistralai/Mixtral-8x7B-Instruct-v0.1", json.load(open('experiments/Mixtral-8x7B-Instruct-v0.1-wikidata.json', 'r'))),
    # ("meta-llama/Llama-2-70b-chat-hf", json.load(open('experiments/Llama-2-70b-chat-hf-wikidata.json', 'r'))),
    ("claude-3-opus-20240229", json.load(open('experiments/claude-3-opus-20240229-wikidata.json', 'r'))),
]

In [56]:
display_summary_stats(wikidata_runs)

,Model,ACC,AUC,F1 Macro,Cohen's kappa
2,gpt-4-0125-preview,0.835793,0.831040,0.831243,0.662487
5,claude-3-opus-20240229,0.813996,0.820338,0.812275,0.626301
3,mistralai/Mistral-7B-Instruct-v0.2,0.769596,0.772741,0.767704,0.536801
4,mistralai/Mixtral-8x7B-Instruct-v0.1,0.758364,0.767966,0.757278,0.519068
1,gpt-3.5-turbo,0.705882,0.698231,0.697979,0.395967
0,google/gemma-7b-it,0.704120,0.678096,0.680917,0.369313


In [57]:
caligraph_runs = [
    ("google/gemma-7b-it", json.load(open('experiments/gemma-7b-it-caligraph.json', 'r'))),
    ("gpt-3.5-turbo", json.load(open('experiments/gpt-3.5-turbo-caligraph.json', 'r'))),
    ("gpt-4-0125-preview", json.load(open('experiments/gpt-4-0125-preview-caligraph.json', 'r'))),
    ("mistralai/Mistral-7B-Instruct-v0.2", json.load(open('experiments/Mistral-7B-Instruct-v0.2-caligraph.json', 'r'))),
    ("mistralai/Mixtral-8x7B-Instruct-v0.1", json.load(open('experiments/Mixtral-8x7B-Instruct-v0.1-caligraph.json', 'r'))),
    # ("meta-llama/Llama-2-70b-chat-hf", json.load(open('experiments/Llama-2-70b-chat-hf-wikidata.json', 'r'))),
    ("claude-3-opus-20240229", json.load(open('experiments/claude-3-opus-20240229-caligraph.json', 'r'))),
]

In [58]:
display_summary_stats(caligraph_runs)

,Model,ACC,AUC,F1 Macro,Cohen's kappa
2,gpt-4-0125-preview,0.948953,0.947859,0.948724,0.897500
5,claude-3-opus-20240229,0.935864,0.935912,0.935758,0.871518
4,mistralai/Mixtral-8x7B-Instruct-v0.1,0.897606,0.896734,0.897238,0.794521
1,gpt-3.5-turbo,0.853403,0.853696,0.853258,0.706567
3,mistralai/Mistral-7B-Instruct-v0.2,0.813333,0.811448,0.810180,0.625192
0,google/gemma-7b-it,0.684350,0.668402,0.640247,0.347324


In [70]:
RESULTS_DIRECTORY = "experiments2"
RESULTS_FILENAME_PATTERN = r"(.+)-kamel-(P.+).json"

In [109]:
files = [file for file in os.listdir(RESULTS_DIRECTORY) if os.path.isfile(os.path.join(RESULTS_DIRECTORY, file))]
matched_files = [file for file in files if re.match(RESULTS_FILENAME_PATTERN, file)]
summary_accs = {}
for file in matched_files:
    m = re.match(RESULTS_FILENAME_PATTERN, file)
    # print(m)
    model = m.group(1)
    property = m.group(2)
    results = json.load(open(os.path.join(RESULTS_DIRECTORY, file), 'r'))
    n = len(results)
    if not property in summary_accs:
        summary_accs[property] = { "Property": results[0]["predicate"], "ID": property, "N": n }
    summary_accs[property][model] = confusion_matrix(results).AUC["1"]
df = pd.DataFrame.from_records(list(summary_accs.values())).sort_values("Property")
df

,Property,ID,N,Mistral-7B-Instruct-v0.2,Meta-Llama-3-70B-Instruct,gpt-3.5-turbo,gpt-4o-2024-05-13,Mixtral-8x7B-Instruct-v0.1
11,adjacent station,P197,264,0.886073,0.727273,0.856061,0.900058,0.900821
15,cause of death,P509,382,0.658163,0.632178,0.589005,0.696335,0.632150
7,chairperson,P488,276,0.844106,0.760870,0.847826,0.851449,0.757571
13,continent,P30,392,0.684332,0.918367,0.650510,0.670918,0.660714
3,filming location,P915,354,0.823150,0.757062,0.683616,0.759887,0.757513
16,headquarters location,P159,296,0.929476,0.915842,0.878378,0.895270,0.914989
0,historic county,P7959,362,0.952861,0.713837,0.831492,0.950276,0.877839
2,inflows,P200,264,0.918254,0.860360,0.867424,0.912879,0.890685
14,influenced by,P737,332,0.708048,0.774194,0.837349,0.903614,0.719532
6,legislated by,P467,384,0.929825,0.687500,0.705729,0.700521,0.691181


In [110]:
df[["N"]].sum()

N    6298
dtype: int64

In [111]:
df_mean = df[[
    "Mistral-7B-Instruct-v0.2", 
    "gpt-3.5-turbo",
    "Mixtral-8x7B-Instruct-v0.1",
    "gpt-4o-2024-05-13",  
    "Meta-Llama-3-70B-Instruct"
]].aggregate(["mean", "max", "min"])
df_mean

,Mistral-7B-Instruct-v0.2,gpt-3.5-turbo,Mixtral-8x7B-Instruct-v0.1,gpt-4o-2024-05-13,Meta-Llama-3-70B-Instruct
mean,0.835700,0.814100,0.816185,0.863692,0.820035
max,0.952861,0.991176,0.988235,0.991176,0.986957
min,0.658163,0.589005,0.632150,0.670918,0.632178


In [105]:
df_model_mean = df[[
    "Mistral-7B-Instruct-v0.2", 
    "gpt-3.5-turbo",
    "Mixtral-8x7B-Instruct-v0.1",
    "gpt-3.5-turbo", 
    "Mixtral-8x7B-Instruct-v0.1"
]].aggregate(["mean", "max", "min"],axis=1)
pd.concat([df[["Property", "ID"]], df_model_mean], axis=1).sort_values("mean", ascending=False)

,Property,ID,mean,max,min
17,school district,P5353,0.978909,0.991176,0.935721
18,licensed to broadcast to,P1408,0.943028,0.956790,0.891304
7,mountain range,P4552,0.938576,0.942434,0.935028
1,parent taxon,P171,0.909103,0.921553,0.900524
15,headquarters location,P159,0.903242,0.929476,0.878378
2,inflows,P200,0.886895,0.918254,0.867424
10,adjacent station,P197,0.879967,0.900821,0.856061
0,historic county,P7959,0.874305,0.952861,0.831492
8,librettist,P87,0.864423,0.950160,0.801205
19,place served by transport hub,P931,0.861074,0.919118,0.744988


In [64]:
ALO_RESULTS_FILENAME_PATTERN = r"(.+)-kamel-alo-(P.+).json"

In [65]:
files = [file for file in os.listdir(RESULTS_DIRECTORY) if os.path.isfile(os.path.join(RESULTS_DIRECTORY, file))]
matched_files = [file for file in files if re.match(ALO_RESULTS_FILENAME_PATTERN, file)]
summary_accs = {}
for file in matched_files:
    m = re.match(ALO_RESULTS_FILENAME_PATTERN, file)
    model = m.group(1)
    property = m.group(2)
    results = json.load(open(os.path.join(RESULTS_DIRECTORY, file), 'r'))
    if not property in summary_accs:
        summary_accs[property] = { "Property": results[0]["predicate"], "ID": property }
    summary_accs[property][model] = confusion_matrix(results).TPR["1"]
alo_df = pd.DataFrame.from_records(list(summary_accs.values())).sort_values("Property")
alo_df

KeyError: 'Property'

In [ ]:
alo_df_mean = alo_df[["Mistral-7B-Instruct-v0.2", "gemma-7b-it", "gpt-4-0125-preview", "gpt-3.5-turbo", "Mixtral-8x7B-Instruct-v0.1"]].aggregate(["mean", "max", "min"])
alo_df_mean

,Mistral-7B-Instruct-v0.2,gemma-7b-it,gpt-4-0125-preview,gpt-3.5-turbo,Mixtral-8x7B-Instruct-v0.1
mean,0.825094,0.904888,0.750090,0.728245,0.664515
max,1.000000,1.000000,0.984848,1.000000,0.937500
min,0.400000,0.713415,0.269841,0.359281,0.375000


In [ ]:
alo_df_model_mean = alo_df[["Mistral-7B-Instruct-v0.2", "gemma-7b-it", "gpt-4-0125-preview", "gpt-3.5-turbo", "Mixtral-8x7B-Instruct-v0.1"]].aggregate(["mean", "max", "min"],axis=1)
pd.concat([alo_df[["Property", "ID"]], alo_df_model_mean], axis=1).sort_values("mean", ascending=False)

,Property,ID,mean,max,min
7,legislated by,P467,0.966346,0.989583,0.937500
14,school district,P5353,0.964055,0.987421,0.928994
16,place served by transport hub,P931,0.963407,1.000000,0.856061
4,competition class,P2094,0.924551,1.000000,0.728814
3,adjacent station,P197,0.858296,1.000000,0.692308
17,participant,P710,0.840394,1.000000,0.692913
8,headquarters location,P159,0.815990,0.993103,0.675676
19,position held,P39,0.803852,0.964286,0.700000
10,parent taxon,P171,0.796251,0.871429,0.686170
15,historic county,P7959,0.792455,0.910112,0.646409
